## Mini lab para procesamiento de PDFs implementando TF-IDF y luego un modelo preentrenado de textos académicos/técnicos y comparando sus resultados

### Librerias requeridas

In [70]:
from langchain.document_loaders import PyPDFLoader
import nltk
from nltk.tokenize import sent_tokenize
import spacy
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import torch
import numpy as np

### Extracción de textos usando la librería PyPDF2

In [71]:
# Descargar recursos de NLTK necesarios
nltk.download('punkt_tab')

# Cargar y dividir el PDF
pdf = "calculo1.pdf"
loader = PyPDFLoader(pdf)
documents = loader.load_and_split()

# Extraer el contenido de texto de los documentos
text_chunks = [sent_tokenize(doc.page_content) for doc in documents]
text_chunks = [sentence for chunk in text_chunks for sentence in chunk]  # Aplanar la lista

print(f"Documentos desde el PDF: {len(documents)}")
print(f"Fragmentos tokenizados: {len(text_chunks)}")


[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\ivill\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


Documentos desde el PDF: 848
Fragmentos tokenizados: 22078


### Preprocesamiento de los fragmentos
1) Eliminar stopwords
2) Eliminar caracteres especiales
3) Convertir a minusculas
4) Eliminar espacios adicionales
5) Tokenizar y analizar con spaCy
6) Filtrar tokens irrelevantes
7) Lematizar palabras
8) Reconstruir el texto limpio
9) Agrupar fragmentos procesados en bloques más largos

In [72]:
# Cargar modelo de spaCy en español
nlp = spacy.load("es_core_news_sm")

# Stopwords dinámicas: union de spaCy y personalizadas
spanish_stopwords = nlp.Defaults.stop_words
custom_stopwords = {"ejemplo", "puede", "también"}
stopwords = spanish_stopwords.union(custom_stopwords)

def preprocess_text(text):
    text = re.sub(r'\W+', ' ', text).lower().strip()
    doc = nlp(text)
    lemmatized_words = [
        token.lemma_ for token in doc
        if token.lemma_ not in stopwords and not token.is_punct and not token.is_space
    ]
    return ' '.join(lemmatized_words)

# Aplicar preprocesamiento a los fragmentos
text_chunks_cleaned = [preprocess_text(chunk) for chunk in text_chunks]

# Agrupar fragmentos en bloques más largos
chunk_size = 3
grouped_chunks = [
    ' '.join(text_chunks_cleaned[i:i+chunk_size])
    for i in range(0, len(text_chunks_cleaned), chunk_size)
]

Aplicar TF-IDF para convertir los fragmentos de texto en vectores dispersos que reflejan la relevancia de las palabras

In [73]:
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(text_chunks)
tfidf_vocab = tfidf_vectorizer.vocabulary_
print(f"TF-IDF matrix shape: {tfidf_matrix.shape}")

TF-IDF matrix shape: (22078, 13618)


Utilizar un modelo preentrenado para generar embeddings densos que capturen el significado semántico de los fragmentos de texto

In [74]:
embedding_model = SentenceTransformer('all-mpnet-base-v2') # Modelo preentrenado optimizado para textos educativos / tecnicos
dense_embeddings = embedding_model.encode(text_chunks, convert_to_tensor=False)
dense_embeddings_array = np.array(dense_embeddings)
print(f"Dense embeddings shape: {dense_embeddings_array.shape}")

Dense embeddings shape: (22078, 768)


Definir funciones para realizar búsquedas semánticas utilizando los vectores generados por TF-IDF y embeddings densos, comparando sus resultados a través de la similitud coseno

In [77]:
def semantic_search_sparse(query, top_k=3):
    query_vector = tfidf_vectorizer.transform([query])
    similarities = cosine_similarity(query_vector, tfidf_matrix).flatten()
    top_indices = similarities.argsort()[-top_k:][::-1]
    return [(text_chunks[i], similarities[i]) for i in top_indices]

def semantic_search_dense(query, top_k=3):
    query_embedding = embedding_model.encode(query, convert_to_tensor=True)
    dense_embeddings_tensor = torch.tensor(dense_embeddings)
    similarities = torch.nn.functional.cosine_similarity(
        query_embedding.unsqueeze(0), dense_embeddings_tensor, dim=-1
    )
    top_indices = torch.topk(similarities, top_k).indices
    return [(text_chunks[i], similarities[i].item()) for i in top_indices]

def compare_results(sparse_results, dense_results):
    print(f"\n{'='*10} Comparación de Resultados Sparse vs Dense {'='*10}")
    for i in range(max(len(sparse_results), len(dense_results))):
        print(f"\nResultado {i + 1}:")
        if i < len(sparse_results):
            print(f"  [TF-IDF] Score: {sparse_results[i][1]:.2f}")
            print(f"  [TF-IDF] Fragmento: {sparse_results[i][0][:300].strip()}...")
        if i < len(dense_results):
            print(f"  [Embeddings] Score: {dense_results[i][1]:.2f}")
            print(f"  [Embeddings] Fragmento: {dense_results[i][0][:300].strip()}...")
        print("-" * 50)


Evaluar el laboratorio

In [78]:
print("Mini lab de Búsqueda Semántica")
query = input(f"\nIntroducir la consulta para el texto {pdf} (o escribe 'exit' para salir): ")
sparse_results = semantic_search_sparse(query)
dense_results = semantic_search_dense(query)
compare_results(sparse_results, dense_results)

Laboratorio de Búsqueda Semántica

========== Comparación de Resultados Sparse vs Dense ==========

Resultado 1:
  [TF-IDF] Score: 0.60
  [TF-IDF] Fragmento: Entonces, la derivada es 
f x x 1
2 x x 1 2....
  [Embeddings] Score: 0.79
  [Embeddings] Fragmento: Encuentre la primera derivada....
--------------------------------------------------

Resultado 2:
  [TF-IDF] Score: 0.58
  [TF-IDF] Fragmento: Por ejemplo, la tercera derivada 
es la derivada de la segunda derivada....
  [Embeddings] Score: 0.77
  [Embeddings] Fragmento: Compruebe su respuesta por derivación....
--------------------------------------------------

Resultado 3:
  [TF-IDF] Score: 0.55
  [TF-IDF] Fragmento: La segunda derivada es un ejemplo de una derivada de orden superior....
  [Embeddings] Score: 0.76
  [Embeddings] Fragmento: Encuentre la segunda derivada....
--------------------------------------------------
